In [1]:
import cv2
import os
import numpy as np

# New Section

In [13]:
def double_disparity_values(disp_map):
    return disp_map * 2

def apply_gaussian_blur(disp_map):
    return cv2.GaussianBlur(disp_map, (5, 5), 0)

def calculate_disparity_region(left_img, right_img, metric, disp_limit, tmpl_width, tmpl_height):
    height, width = left_img.shape
    disparity_map = np.zeros_like(left_img, dtype=np.float32)
    half_width = tmpl_width // 2
    half_height = tmpl_height // 2
    search_range = range(-disp_limit, disp_limit + 1)

    for y in range(half_height, height - half_height):
        for x in range(half_width, width - half_width):
            lowest_cost = float('inf')
            optimal_shift = 0
            for shift in search_range:
                x_r = x - shift
                if x_r < half_width or x_r >= width - half_width:
                    continue
                cost = compute_cost(left_img, right_img, x, y, x_r, metric, tmpl_width, tmpl_height)
                if cost < lowest_cost:
                    lowest_cost = cost
                    optimal_shift = shift
            disparity_map[y, x] = optimal_shift

    return disparity_map


def compute_cost(left_img, right_img, x, y, x_r, metric, width, height):
    patch_l = left_img[y - height//2:y + height//2 + 1, x - width//2:x + width//2 + 1]
    patch_r = right_img[y - height//2:y + height//2 + 1, x_r - width//2:x_r + width//2 + 1]

    if metric == 'SAD':
        return np.sum(np.abs(patch_l - patch_r))
    elif metric == 'SSD':
        return np.sum((patch_l - patch_r) ** 2)
    elif metric == 'NCC':
        avg_l = np.mean(patch_l)
        avg_r = np.mean(patch_r)
        numerator = np.sum((patch_l - avg_l) * (patch_r - avg_r))
        denominator = np.sqrt(np.sum((patch_l - avg_l) ** 2) * np.sum((patch_r - avg_r) ** 2))
        return -numerator / denominator if denominator != 0 else float('inf')

def compute_feature_based_disparity(left_img, right_img, search_range, disparity):
    corners_l = cv2.cornerHarris(left_img, 2, 3, 0.04)
    corners_r = cv2.cornerHarris(right_img, 2, 3, 0.04)

    keypoints_l = cv2.goodFeaturesToTrack(corners_l, maxCorners=100, qualityLevel=0.01, minDistance=10)
    keypoints_r = cv2.goodFeaturesToTrack(corners_r, maxCorners=100, qualityLevel=0.01, minDistance=10)

    if keypoints_l is not None and keypoints_r is not None:
        keypoints_l = keypoints_l.reshape(-1, 2)
        keypoints_r = keypoints_r.reshape(-1, 2)
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
        matches = bf.match(keypoints_l, keypoints_r)

        for match in matches:
            query_idx = match.queryIdx
            train_idx = match.trainIdx
            if query_idx < len(keypoints_l) and train_idx < len(keypoints_r):
                x, y = map(int, keypoints_l[query_idx])
                x_r = int(keypoints_r[train_idx][0])
                if 0 <= x < disparity.shape[1] and 0 <= y < disparity.shape[0] and 0 <= x_r < right_img.shape[1]:
                    disparity[y, x] = np.abs(x - x_r)

    return disparity


def filter_disparity(disp_map):
    return (disp_map > 0).astype(np.uint8)

def fill_disparity_gaps(disp_map):
    zero_mask = (disp_map == 0).astype(np.uint8)
    return cv2.inpaint(disp_map.astype(np.uint8), zero_mask, 3, cv2.INPAINT_TELEA)

def process_disparity():
    set_name = input("Enter the name of the image set: ")

    image_path_map = {
        'sawtooth': ('sawtoothim0.ppm', 'sawtoothim8.ppm'),
        'venus': ('venusim0.ppm', 'venusim8.ppm'),
        'bull': ('bullim0.ppm', 'bullim8.ppm'),
        'poster': ('posterim0.ppm', 'posterim8.ppm'),
        'barn1': ('barn1im0.ppm', 'barn1im8.ppm')
    }

    if set_name not in image_path_map:
        raise ValueError("Unknown image set name")

    left_image_path, right_image_path = image_path_map[set_name]
    left_image = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)

    if left_image is None or right_image is None:
        print("Error: Check the image paths.")
        return

    levels = int(input('Enter number of levels: '))
    if levels <= 0:
        raise ValueError('Number of levels must be greater than zero.')

    disparity = np.zeros_like(left_image, dtype=np.float32)

    for level in range(levels):
      print(f'Processing level {level + 1}')
      metric = input('Enter distance metric [SAD, SSD, NCC]: ')
      method = input('Enter method [region, feature]: ')
      range_limit = int(input('Search range: '))
      tmpl_w = int(input('Template width (odd number): '))
      tmpl_h = int(input('Template height (odd number): '))

      if level == 0:
          disparity = np.zeros_like(left_image, dtype=np.float32)
      else:
          disparity = double_disparity_values(disparity)

      if method == 'region':
          disparity = calculate_disparity_region(left_image, right_image, metric, range_limit, tmpl_w, tmpl_h)
      elif method == 'feature':
          disparity = compute_feature_based_disparity(left_image, right_image, range_limit, disparity)

      disparity = apply_gaussian_blur(disparity)
      disparity_normalized = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
      cv2.imwrite(f'disparity_level_{level + 1}.png', disparity_normalized.astype(np.uint8))

    valid_disparity = filter_disparity(disparity)
    final_disparity = fill_disparity_gaps(disparity)

    disparity_normalized_final = cv2.normalize(valid_disparity * final_disparity + (1 - valid_disparity) * disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

if __name__ == "__main__":
    process_disparity()


Enter the name of the image set: poster
Enter number of levels: 3
Processing level 1
Enter distance metric [SAD, SSD, NCC]: SAD
Enter method [region, feature]: region
Search range: 33
Template width (odd number): 3
Template height (odd number): 3
Processing level 2
Enter distance metric [SAD, SSD, NCC]: SSD
Enter method [region, feature]: feature
Search range: 35
Template width (odd number): 5
Template height (odd number): 5
Processing level 3
Enter distance metric [SAD, SSD, NCC]: NCC
Enter method [region, feature]: region
Search range: 39
Template width (odd number): 3
Template height (odd number): 3
